In [1]:
# ostatnie piwo: https://www.polskikraft.pl/piwo/9740
# ostatni browar: https://www.polskikraft.pl/browar/411
# ostatni user: https://www.polskikraft.pl/profil/28348

# imports

In [2]:
import os
dumpdir = '/content/drive/MyDrive/datasets/PolskiKraft'

In [3]:
import pandas as pd
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

In [ ]:
# import the data
!cp {dumpdir}/*.zip .
!unzip -q piwa.zip
!unzip -q browary.zip

# Extracting the data

## Etractors (defining the functions)

### Brewery

In [5]:
with open('browary/411.html', 'r') as f:
    html_content = f.read()

In [6]:
def extract_brewery_beers_from_brewery_html(html_content):
    #in: string containing html file
    #out: list of dicts of extracted properties:
    #     [{...}]
    soup = BeautifulSoup(html_content, 'html.parser')
    beer_entries = soup.find_all("div", {"class": "pk-search-entry-link"})
    results = []
    for html_entry in beer_entries:

        tmp = html_entry.find_all("div", {"class":"row pk-search-entry"})[0].contents
        results.append({
            'brewery_name': soup.find_all("ol", {"class":"breadcrumb pk-breadcrumb"})[0].find_all('li')[1].text,
            'name': tmp[3].text,
            'rating': float(html_entry.find_all("div", {"class":"row rating"})[0].text),
            'type': tmp[6].text,
            'thumbnail_link': tmp[1].get('src'),
        })
    return results

In [7]:
pd.DataFrame(extract_brewery_beers_from_brewery_html(html_content))

,brewery_name,name,rating,type,thumbnail_link
0,Monkey Browar,Wróg U Bram,3.6,American Pale Ale,https://www.polskikraft.pl/img/photos/16019238...
1,Monkey Browar,Born To Be Hoppy,4.1,DDH DIPA,https://www.polskikraft.pl/img/photos/16019237...
2,Monkey Browar,Born To Be Hoppy,4.4,DDH Double IPA,https://www.polskikraft.pl/img/photos/16019229...


In [8]:
def extract_brewery_rating_from_brewery_html(html_content):
    #in: string containing html file
    #out: dict of extracted properties:
    #     {...}
    soup = BeautifulSoup(html_content, 'html.parser')
    the_div = soup.find_all("div", {"class":"panel-body pk-panel-photo-tile"})[0]
    results = {
        'name': the_div.find("p", {"class":"beer-name"}).text,
        'beers': the_div.find("p", {"class":"brewery-name"}).text,
        'rating': float(the_div.find("span", {"class":"rating"}).text),
        'logo_link': soup.find_all("div", {"class":"panel-body pk-panel-photo-tile"})[0].find('img').get('src'),
    }
    return results

In [9]:
extract_brewery_rating_from_brewery_html(html_content)

{'beers': '3 piwa',
 'logo_link': 'https://www.polskikraft.pl/img/photos/1601922522.jpg',
 'name': 'Monkey Browar',
 'rating': 4.0}

### Beers

In [10]:
with open('piwa/9740.html', 'r') as f:
    html_content = f.read()

In [11]:
def extract_similar_beers_from_beer_html_soup(soup):
    #in: string containing html file
    #out: dict of extracted properties:
    #     {...}

    similar_div = soup.find_all("div", {"class":"panel-body pk-panel-tile pk-panel-tile-list"})[4]
    hrefs = [x.get("href") for x in similar_div.find_all("a", {"class":"big-link"})]
    return [{'similar': x.split('_')[-1], 'similar_link': x} for x in hrefs]

def extract_similar_beers_from_beer_html(html_content):
    #in: string containing html file
    #out: dict of extracted properties:
    #     {...}

    soup = BeautifulSoup(html_content, 'html.parser')
    return extract_similar_beers_from_beer_html_soup(soup)

In [12]:
pd.DataFrame(extract_similar_beers_from_beer_html(html_content))

,similar,similar_link
0,1748,https://www.polskikraft.pl/piwo/pinta-vermont-...
1,2330,https://www.polskikraft.pl/piwo/browar-ziemia-...
2,2476,https://www.polskikraft.pl/piwo/browar-bazylis...
3,2485,https://www.polskikraft.pl/piwo/browar-deer-be...
4,2568,https://www.polskikraft.pl/piwo/piwne-podziemi...
...,...,...
294,9700,https://www.polskikraft.pl/piwo/piwojad-test-%...
295,9704,https://www.polskikraft.pl/piwo/browar-stu-mos...
296,9705,https://www.polskikraft.pl/piwo/piwojad-ddh-ne...
297,9723,https://www.polskikraft.pl/piwo/alebrowar-new-...


In [13]:
def extract_beer_data_from_beer_html_soup(soup):
    #in: string containing html file
    #out: dict of extracted properties:
    #     {...}

    the_div = soup.find("div", {"class":"panel-body pk-panel-photo-tile"})
    description_div = soup.find("div", {"class":"panel-body pk-panel-tile"})
    results = {
        'name': the_div.find("p", {"class":"beer-name"}).text,
        'brewery': the_div.find("p", {"class":"brewery-name"}).text,
        'rating': the_div.find("span", {"class":"rating"}).text.strip(),
        'logo_link': soup.find("div", {"class":"panel-body pk-panel-photo-tile"}).find('img').get('src'),
        'features': {x.find("p").text: x.find("h1").text
            for x in description_div.find_all("div", {"class": "col-xs-4"})},
        'descriptive_features': {x.find("h3").text: x.find("p").text.strip()
            for x in description_div.find_all("div", {"class": "col-xs-12 pk-details-category"})}
    }
    return results

def extract_beer_data_from_beer_html(html_content):
    #in: string containing html file
    #out: dict of extracted properties:
    #     {...}

    soup = BeautifulSoup(html_content, 'html.parser')
    return extract_beer_data_from_beer_html_soup(soup)

In [14]:
extract_beer_data_from_beer_html(html_content)

{'brewery': 'BROWAR ABSZTYFIKANT',
 'descriptive_features': {'CHMIELE': 'Citra (USA), Mosaic (USA), Simcoe (USA)',
  'STYL': 'New England IPA'},
 'features': {'alc': '5.6', 'blg': '14', 'ibu': '-'},
 'logo_link': 'https://www.polskikraft.pl/img/photos/1607626847.jpg',
 'name': 'Jupikajej 6.0',
 'rating': '3.1'}

In [15]:
def extract_user_ratings_from_beer_html_soup(soup):
    #in: string containing html file
    #out: list of dicts of extracted properties:
    #     {...}
    the_div = soup.find("div", {"class":"panel panel-default hidden-xs"})
    ratings_element = the_div.find_all("div", {"class":"row pk-comments-item pk-comment-link"})
    results = []
    for user_rating_element in ratings_element:
        user_profile_link = user_rating_element.find_all("a")[1].get('href')
        username_element = user_rating_element.find("p", {"class": "username"})
        tmp = user_rating_element.find_all("div", {"class":"row pk-comments-bjcp-bar"})
        results.append({
            'rating': len(user_rating_element.find_all("i", {"class":"fa fa-star fa-lg"})),
            'user': list(username_element.children)[0].strip(),
            'date': username_element.find("sub").text,
            'user_id': user_profile_link.split('/')[-1],
            'user_profile': user_profile_link,
            'user_avatar_link': user_rating_element.find("img").get("src"),
            'comment': user_rating_element.find("p", {"class":"content"}).text.strip(),
            'BJCP': {
                x.find('p', {"class":"title"}).text:
                x.find('p', {"class":"rating"}).text
                for x in tmp
                },
        })
    return results

def extract_user_ratings_from_beer_html(html_content):
    #in: string containing html file
    #out: list of dicts of extracted properties:
    #     {...}
    soup = BeautifulSoup(html_content, 'html.parser')
    return extract_user_ratings_from_beer_html_soup(soup)

In [16]:
pd.DataFrame(extract_user_ratings_from_beer_html(html_content))

,rating,user,date,user_id,user_profile,user_avatar_link,comment,BJCP
0,3,eMeS,12/02/2021,2201,https://www.polskikraft.pl/profil/2201,https://graph.facebook.com/v2.5/13347658665491...,,{}
1,3,Tomasz Romaszko,13/12/2020,21290,https://www.polskikraft.pl/profil/21290,https://graph.facebook.com/v2.5/22950427405543...,,{}
2,3,WaldoBeerSan,12/12/2020,14866,https://www.polskikraft.pl/profil/14866,https://www.polskikraft.pl/img/layout/ic_user_...,"Dobre Piwo, słodycz niska, barwa bursztynowa, ...","{'AROMAT': '8/12', 'WYGLĄD': '3/3', 'SMAK': '1..."
3,3,Piter Casch,10/12/2020,13459,https://www.polskikraft.pl/profil/13459,https://platform-lookaside.fbsbx.com/platform/...,Wygląd wody z kałuży. W smaku ok.,{}


## Creating the dataset

### Beers

In [17]:
# old bad_files = [63, 65, 66, 130, 279, 383, 490, 1335, 1350, 1462, 1674, 1699, 1705, 1725, 1750, 1764, 2331, 2337, 2364, 2722, 2748, 2810, 3036, 3645, 3745, 4046, 4221, 4434, 4931, 6044, 6148, 6170, 6670, 6870, 6950, 7336]

In [18]:
beers_metadata = []
PK_ratings = []
beer_network_according_to_PK = []
bad_files = []

for i in tqdm(range(1,9740+1)):
    try:
        path = f'piwa/{i}.html'
        with open(path, 'r') as f:
            html_content = f.read()

        soup = BeautifulSoup(html_content, 'html.parser')
        metadata = extract_beer_data_from_beer_html_soup(soup)
        ratings = extract_user_ratings_from_beer_html_soup(soup)
        similar_to = extract_similar_beers_from_beer_html_soup(soup)
        edges = [(i,x['similar']) for x in similar_to]

        # update data with beer id
        metadata.update({'beer_id': i})
        for r in ratings:
            r.update({'beer_id': i})
        beers_metadata.append(metadata)

        # unpack specific features
        mf = metadata.pop('features')
        mdf = metadata.pop('descriptive_features')
        metadata.update(mf)
        metadata.update(mdf)
        for r in ratings:
            bjcp_features = r.pop('BJCP')
            r.update(bjcp_features)

        # append the entries
        PK_ratings += ratings
        beer_network_according_to_PK += edges
    except:
        bad_files.append(i)

beers_metadata = pd.DataFrame(beers_metadata)
PK_ratings = pd.DataFrame(PK_ratings)
print('bad_files:', bad_files)


bad_files: [63, 65, 66, 130, 383, 490, 1335, 1350, 1462, 1674, 1699, 1705, 1725, 1750, 1764, 2331, 2337, 2364, 2722, 2748, 2810, 3036, 3645, 3745, 4046, 4221, 4434, 4931, 6044, 6148, 6170, 6670, 6870, 6950, 7336]


In [19]:
len(bad_files)

35

In [20]:
beers_metadata.head()

,name,brewery,rating,logo_link,beer_id,blg,alc,ibu,STYL,CHMIELE,SŁODY,DROŻDŻE,KONTRAKTOR,INNE DODATKI
0,Rowing Jack,ALEBROWAR,4.2,https://www.polskikraft.pl/img/photos/14422467...,1,16,6.2,70,American India Pale Ale,"Simcoe (USA), Chinook (USA), Citra (USA), Casc...","Pale Ale, Pszeniczny, Wiedeński, Carapils®",Safale US-05,Gościszewo,NaN
1,70 Szylingów,PINTA,3.5,https://www.polskikraft.pl/img/photos/14429079...,2,-,3.2,-,Scottish Ale 70,East Kent Goldings (GB),"Pale Ale Weyermann®, Carapils®, Wędzony Wayerm...",Safbrew S-33,Pinta,NaN
2,a ja pale ale,PINTA,3.7,https://www.polskikraft.pl/img/photos/14422597...,3,12,4.5,-,American Pale Ale,"Columbus (USA), Chinook (USA), Cascade (USA), ...","Pale Ale Weyermann®, Diastatyczny jęczmieny We...",Safale US-05,Pinta,NaN
3,a'la Grodziskie,PINTA,3.2,https://www.polskikraft.pl/img/photos/14422607...,4,7.8,2.6,-,Grodziskie,Lubelski (PL),Pszeniczny wędzony,Safbrew S-33,Pinta,NaN
4,Ale Szycha,PINTA,4.2,https://www.polskikraft.pl/img/photos/14423453...,5,13.1,5,-,Polskie Ale,"Marynka szyszka (PL), Lubelski szyszka (PL)",Pilzneńśki Premium Weyermann,Fermentis S-04,Pinta,NaN


In [21]:
PK_ratings.head()

,rating,user,date,user_id,user_profile,user_avatar_link,comment,beer_id,AROMAT,WYGLĄD,SMAK,ODCZUCIE W USTACH,OGÓLNE WRAŻENIE
0,4,Piwnypolitolog.pl,02/03/2016,1147,https://www.polskikraft.pl/profil/1147,https://graph.facebook.com/v2.5/10561137277617...,"Ocena nieco uśredniona, bo różne warki się tra...",1,9/12,3/3,14/20,4/5,7/10
1,5,mojepiwa.pl,18/05/2016,1967,https://www.polskikraft.pl/profil/1967,https://platform-lookaside.fbsbx.com/platform/...,"Powrót do klasyka, którego ma się w pamięci ja...",1,NaN,NaN,NaN,NaN,NaN
2,5,Chmielokracja,20/09/2015,229,https://www.polskikraft.pl/profil/229,https://graph.facebook.com/v2.5/99666886371456...,,1,NaN,NaN,NaN,NaN,NaN
3,3,Damian Popiela,27/02/2021,28165,https://www.polskikraft.pl/profil/28165,https://platform-lookaside.fbsbx.com/platform/...,,1,NaN,NaN,NaN,NaN,NaN
4,4,Mateusz Mioduszewski,10/02/2021,26727,https://www.polskikraft.pl/profil/26727,https://graph.facebook.com/v2.5/32551460645640...,,1,NaN,NaN,NaN,NaN,NaN


save the results to a file

In [22]:
beers_metadata.to_csv('beers_metadata.csv', index=False)

In [23]:
PK_ratings.to_csv('ratings.csv', index=False)

In [24]:
with open('beer_network.csv', 'w') as f:
    for x in beer_network_according_to_PK:
        f.write(f"{x[0]}, {x[1]}\n")

In [25]:
!cp beers_metadata.csv ratings.csv beer_network.csv {dumpdir}

In [26]:
!cp beers_metadata.csv ratings.csv beer_network.csv /content/drive/MyDrive/DANsem3/SR